In [1]:
import anthropic
client = anthropic.Anthropic(api_key="")

In [ ]:
import os
for name in os.listdir('docdb/CRV'):
    if name == ".ipynb_checkpoints":
        continue
    
    # Open file
    else:
        x = ""
        with open(os.path.join('docdb/CRV', name)) as f:
            print(f"Content of '{name}'")
            x +=f"Content of '{name}'"

    
            path, extension = os.path.splitext(name)

            if extension != ".pdf" or extension != ".txt":
                continue
           
            else:
                if extension == ".pdf":
                    text = extract_text('docdb/CRV/'+name, laparams=LAParams())
                    print(text)
                    x += text
        
                elif extension == ".txt":
                    print(f.read())
                    text = f.read()
                    x += text
        

                print(f"Summary of '{name}':")
    
                q = f"Summarize this document: {x}"
                message = client.messages.create(
                    model = "claude-3-5-sonnet-20240620",
                    max_tokens = 512,
                    messages = [
                        {"role":"user", "content": q }
                    ]
                )
                print(message.content[0].text)

In [3]:
import re
import PyPDF2
from pptx import Presentation

def convert_pptx(path):

    prs = Presentation(path)
    
    # text_runs will be populated with a list of strings,
    # one for each text run in presentation
    text_runs = []
    
    for slide in prs.slides:
        for shape in slide.shapes:
            if not shape.has_text_frame:
                continue
            for paragraph in shape.text_frame.paragraphs:
                for run in paragraph.runs:
                    text_runs.append(run.text)
    text = ""

    for x in text_runs:
        text += x + '\n'

    return text

def clean_text(text):
    # Remove extra whitespace
    text = ' '.join(text.split())
    
    # Remove page numbers
    text = re.sub(r'\n\s*\d+\s*\n', '\n', text)
    
    # Remove headers and footers (adjust patterns as needed)
    text = re.sub(r'^\s*(.+\n){1,2}', '', text, flags=re.MULTILINE)
    text = re.sub(r'\n(.+\n){1,2}\s*$', '\n', text, flags=re.MULTILINE)
    
    # Replace multiple newlines with a single newline
    text = re.sub(r'\n+', '\n', text)
    
    # Remove bullet points and numbering
    text = re.sub(r'^\s*[\u2022\u2023\u25E6\u2043\u2219]\s*', '', text, flags=re.MULTILINE)
    text = re.sub(r'^\s*\d+\.\s*', '', text, flags=re.MULTILINE)
    
    return text.strip()

def process_pdf(pdf_path):
    raw_text = convert_pptx(pdf_path)
    cleaned_text = clean_text(raw_text)
    return cleaned_text


def summarize(text):
    q = f"Summarize this document in two to five sentences: {text}"
    message = client.messages.create(
        model = "claude-3-5-sonnet-20240620",
        max_tokens = 512,
        messages = [
             {"role":"user", "content": q }
                ]
        )
    return message.content[0].text
    

# Usage
#pdf_path = 'docdb/49745_Elect 7.9.24.pdf'
#processed_text = process_pdf(pdf_path)
#print(processed_text)

In [ ]:
summaries = []
for name in os.listdir('docdb/CRV'):
    if name == ".ipynb_checkpoints":
        continue
    
    path, extension = os.path.splitext(name)
    
    # Open file
    if extension == ".pptx":
        x = ""
        with open(os.path.join('docdb/CRV', name)) as f:
            pdf_path = 'docdb/CRV/'+name
            print(f"Summary of '{name}:'")
            processed_text = process_pdf(pdf_path)
            summary = summarize(processed_text)
            summaries.append(f"Summary of '{name}:'" + summary)

In [ ]:
print(summaries)

In [6]:
from transformers import AutoTokenizer
from angle_emb import AnglE
import json
angle = AnglE.from_pretrained('WhereIsAI/UAE-Large-V1', pooling_strategy='cls').cuda()


/Users/annabelleboots/claude-env/lib/python3.12/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [7]:
import csv
from datetime import datetime

def get_document_date(document_title):
    # Extract the first 5 numbers from the document title
    doc_number = document_title[:5]
    
    try:
        with open('docdb/CRV/docdb dump - CRV electronics-2.csv', 'r') as csvfile:
            csv_reader = csv.DictReader(csvfile)
            
            for row in csv_reader:
                if row['number'] == doc_number:
                    # Assuming the date is in 'YYYY-MM-DD' format in the CSV
                    return row['Date']
                    #return datetime.strptime(row['Date'], '%m-%d-%Y %H:%M:%S').date()   
                    
    except FileNotFoundError:
        print(f"Error: CSV file not found at {csv_file_path}")
    except KeyError as e:
        print(f"Error: Required column {e} not found in the CSV file")
    except ValueError:
        print(f"Error: Invalid date format in the CSV file for document number {doc_number}")
    
    # Return None if no match is found or if there's an error
    return None

# Example usage
#document_title = "28474_Module_tests_plan.pptx"

#date = get_document_date(document_title)
#if date:
#   print(f"Document date: {date}")
#else:
#   print("No matching date found or an error occurred.")



In [8]:
summaries = []
for name in os.listdir('docdb/CRV'):
    if name == ".ipynb_checkpoints":
        continue
    
    path, extension = os.path.splitext(name)
    
    # Open file
    if extension == ".pptx":
        x = ""
        with open(os.path.join('docdb/CRV', name)) as f:
            pdf_path = 'docdb/CRV/'+name
            processed_text = process_pdf(pdf_path)
            summary = summarize(processed_text)
            date = get_document_date(name)
            summaries.append(
                {"name":name, "date":date, "summary":summary, "full text":processed_text}
            )


tokenizer = AutoTokenizer.from_pretrained("WhereIsAI/UAE-Large-V1")
angle = AnglE.from_pretrained('WhereIsAI/UAE-Large-V1', pooling_strategy='cls').cuda()
for document in summaries:
    #print(chunk['page'] + " - "+chunk['title'], end="\r")
    text = "# "+document['name']+"\n"+\
                document['date']+"\n"+\
                document['summary']
    #did not include full text in embedding vector
    document["embedding"] = angle.encode([text])[0].tolist()


if os.path.exists('docdb/CRV/crv_ppts.json') == False:
    open('docdb/CRV/crv_ppts.json', 'x')


with open('docdb/CRV/crv_ppts.json', 'w') as file:
     json.dump(summaries, file, indent=4)

INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 529 "
INFO:anthropic._base_client:Retrying request to /v1/messages in 0.893578 seconds
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/me